### selecting and statistics of the reserviors in hma region

In [9]:
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.ops import unary_union


#### 1. Select the reserviors in hma from global dataset by location.


In [10]:
path_hma = 'data/hma-extent/HMA/hma_gtng_202307_subregions.gpkg'
path_reservoirs = 'data/water-extent/reservoirs/GDW_reservoirs_v1_0.gpkg' 


In [11]:
## Read data
global_reservoirs = gpd.read_file(path_reservoirs)
hma_gpd = gpd.read_file(path_hma)
print(global_reservoirs.shape)
global_reservoirs.head(2)


(35295, 72)


,GDW_ID,RES_NAME,DAM_NAME,ALT_NAME,DAM_TYPE,LAKE_CTRL,RIVER,ALT_RIVER,MAIN_BASIN,SUB_BASIN,...,LONG_DAM,LAT_DAM,ORIG_SRC,POLY_SRC,GRAND_ID,HYRIV_ID,INSTREAM,HYLAK_ID,HYBAS_L12,geometry
0,1,Lake Winnipeg,Jenpeg,Split Lake,Lake Control Dam,Yes,Nelson,None,None,None,...,0.0,0.0,GRanD,CanVec,709,70125969,Instream,4,7120921060,"MULTIPOLYGON (((-98.80636 53.88021, -98.80578 ..."
1,2,Ontario,Iroquois,None,Lake Control Dam,Yes,St. Lawrence,None,None,None,...,0.0,0.0,GRanD,SWBD,1485,70444883,Instream,7,7121021260,"MULTIPOLYGON (((-79.09167 43.81213, -79.09133 ..."


In [12]:
## select the reserviors in the hma region
poly_hma = unary_union(hma_gpd.to_crs('EPSG:3857')['geometry'])
reservoirs_id = np.where(poly_hma.contains(global_reservoirs.to_crs('EPSG:3857')['geometry'].centroid) == True)[0]
hma_reservoirs = global_reservoirs.iloc[reservoirs_id] 
hma_reservoirs = hma_reservoirs.reset_index()
del hma_reservoirs['index']
print(hma_reservoirs.shape)


(485, 72)


In [39]:
# # ### Write out
hma_reservoirs.to_file('data/water-extent/reservoirs/hma_GDW_reservoirs.gpkg', driver='GPKG')


#### 1. statistics of the reservoirs in the hma region.

In [38]:
# Spatial join: assign each reservoir to a subregion polygon
hma_reservoirs_sub = gpd.sjoin(
  hma_reservoirs,
  hma_gpd[['o2region', 'full_name', 'geometry']],
  how='left',
  predicate='within'
)
hma_reservoirs_sub.head(2)

,GDW_ID,RES_NAME,DAM_NAME,ALT_NAME,DAM_TYPE,LAKE_CTRL,RIVER,ALT_RIVER,MAIN_BASIN,SUB_BASIN,...,POLY_SRC,GRAND_ID,HYRIV_ID,INSTREAM,HYLAK_ID,HYBAS_L12,geometry,index_right,o2region,full_name
0,287,Toktogul'skoye,Toktogul,None,Dam,None,Naryn,None,None,None,...,SWBD,4675,40300163,Instream,1306,4120395430,"MULTIPOLYGON (((72.89472 41.84669, 72.89417 41...",2.0,13-03,West Tien Shan
1,292,Kayrakkumskoye,Kayrakkum,Kairakkum,Dam,None,Syr-Daria,None,None,None,...,SWBD,4686,40343454,Instream,1320,4120446290,"MULTIPOLYGON (((70.22608 40.42086, 70.22856 40...",0.0,13-01,Hissar Alay


In [40]:
stat = hma_reservoirs_sub.groupby(['o2region']).agg(
    reservoirs_num=('GDW_ID', 'size'),  # Use 'index_right' to count reservoirs
    reservoirs_area=('AREA_POLY', 'sum')  # Sum the area of reservoirs
).reset_index()


,o2region,full_name,reservoirs_num,reservoirs_area,geometry
0,13-01,Hissar Alay,19.0,563.086,"MULTIPOLYGON (((70 40.7, 71 40.7, 72.01 40.7, ..."
1,13-02,Pamir (Safed Khirs / West Tarim),3.0,77.176,"MULTIPOLYGON (((74.35547 39.80418, 74.37581 39..."
2,13-03,West Tien Shan,57.0,534.176,"MULTIPOLYGON (((77.99937 43.69747, 78.5 43.749..."
3,13-04,East Tien Shan (Dzhungaria),63.0,208.230,"MULTIPOLYGON (((78.5 43.74989, 78.5 44, 78.5 4..."
4,13-05,West Kun Lun,5.0,14.514,"MULTIPOLYGON (((76.49466 37.98237, 76.50852 37..."
5,13-06,East Kun Lun (Altyn Tagh),10.0,60.153,"MULTIPOLYGON (((98.55079 35.27462, 98.51748 35..."
6,13-07,Qilian Shan,55.0,159.506,"MULTIPOLYGON (((104.8 40.5, 104.8 40, 104.8 39..."
7,13-08,Inner Tibet,4.0,7.528,"MULTIPOLYGON (((85.98118 28.91069, 85.97865 28..."
8,13-09,Southeast Tibet,31.0,103.841,"MULTIPOLYGON (((98.55079 35.27462, 98.63497 35..."
9,14-01,Hindu Kush,2.0,15.674,"MULTIPOLYGON (((69.87072 36.87288, 69.88882 36..."


In [ ]:
hma_reservoir_stat = hma_gpd.merge(stat, on='o2region', how='left')
hma_reservoir_stat = hma_reservoir_stat[['o2region', 'full_name', 'reservoirs_num', 'reservoirs_area', 'geometry']]
hma_reservoir_stat = hma_reservoir_stat.fillna(0)
hma_reservoir_stat.head()
# hma_reservoir_stat.to_file('data/water-extent/reservoirs/hma_GDW_reservoirs_stat.gpkg', driver='GPKG')  
